In [13]:
%load_ext autoreload
%autoreload 2
#%pip list

import sys
#from gensim.utils import tokenize
import pandas as pd
import numpy as np
import re
import transformers
from sklearn.preprocessing import LabelEncoder
import os
import re
from utils import *
import plotly.express as px
import plotly.graph_objects as go
#import bert
import torch
import torch.nn as nn
#import gensim
import sys
import string
import logging
import torch.optim as optim
import random
import torchtext

#import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%pip index versions torchtext

torchtext (0.18.0)
Available versions: 0.18.0, 0.17.2, 0.16.2, 0.6.0, 0.5.0, 0.4.0, 0.3.1, 0.2.3, 0.2.1, 0.2.0, 0.1.1
  INSTALLED: 0.17.2
  LATEST:    0.18.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip list

Package                 Version
----------------------- -----------
absl-py                 2.1.0
annotated-types         0.7.0
asttokens               2.4.1
astunparse              1.6.3
blis                    1.0.1
catalogue               2.0.10
certifi                 2024.8.30
charset-normalizer      3.4.0
click                   8.1.7
cloudpathlib            0.20.0
colorama                0.4.6
comm                    0.2.2
confection              0.1.5
contourpy               1.3.1
cycler                  0.12.1
cymem                   2.0.10
debugpy                 1.8.9
decorator               5.1.1
en_core_web_sm          3.8.0
executing               2.1.0
filelock                3.16.1
flatbuffers             24.3.25
fonttools               4.55.0
fsspec                  2024.10.0
gast                    0.6.0
google-pasta            0.2.0
grpcio                  1.68.0
h5py                    3.12.1
huggingface-hub         0.26.3
idna                    3.10
ipykernel     

In [16]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset

class customTextDataset(Dataset):
    def __init__(self, path, colnames, vocab=None):
        self.data =  pd.read_csv(path, names=colnames, header=None, sep = ',', index_col=False)
        self.vocab = vocab
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #To do{
        # indent below with if vocab == None: return sentence, label
        # if vocab is Not none, return tokens, labels
        #}
        sentence = remove_tags(self.data.loc[idx].text)
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
        sentence = re.sub(r'\s+', ' ', sentence)
        sentence = sentence.lower()
        label = self.data.loc[idx].food
        return sentence,label
 
    def getHead(self):
        print(self.data.head())

In [17]:
from torch.utils.data import DataLoader
trainData = customTextDataset(path = 'train_titles.csv', colnames=['image_path', 'text', 'food'])
trainLoader = DataLoader(trainData,batch_size=5)
#trainData.getHead()
testData = customTextDataset(path = 'test_titles.csv', colnames=['image_path', 'text', 'food'])
testLoader = DataLoader(testData)

### Sequence Length Histograms

In [18]:
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
import io
import spacy
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter, OrderedDict

en_tokenizer = get_tokenizer('spacy', language='en')
try:
    spacy_en = spacy.load("en_core_web_sm")
except IOError:
    os.system("python -m spacy download en_core_web_sm")
    spacy_en = spacy.load("en_core_web_sm")

vocabIterator = build_vocab_from_iterator([en_tokenizer(str(string_)) for string_,_ in trainLoader],specials=["<pad>"])
vocabIterator.set_default_index(vocabIterator.get_stoi()["<pad>"])
vocabSize = (vocabIterator.__len__() )
print(f"Vocabulary Size = {vocabSize}")

#def build_vocab_from_counter(data, tokenizer):
#    counter = Counter()
#    for string_ in data:
#        counter.update(tokenizer(str(string_)))
#    
#    sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
#    ordered_dict = OrderedDict(sorted_by_freq_tuples)
#    return vocab(ordered_dict=ordered_dict, specials=["<blank>"])


#vocab = build_vocab_from_counter(processed_train, en_tokenizer)
#vocabSize = (vocab.__len__() )
#print(vocabSize)

c:\virtual Environments\food101\Lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


Vocabulary Size = 19548


### Map Captions to Tokens to IDs and Assign Labels

In [19]:
def token2idNew(listData, maxLen, vocab, tokenizer):
    ids = torch.zeros( (len(listData),maxLen) )
    # set equal to id of 0/pad vector
    pad_id = vocab.get_stoi()["<pad>"]
    print(f"Pad Id = {pad_id}")
    ids[:,:] = torch.tensor(pad_id)
    for i in range(len(listData)):
        indices = torch.tensor(vocab(tokenizer(str(listData[i]))))
        L = min(len(indices),maxLen)
        ids[i,:L]  = indices[:L]
    return ids

In [26]:
numClasses = 101
# optionally downsample by reducing the number of classes 
#train,test = downSampleData(numClasses, train, test)
#countsTrn,countsTst  = classCounts(numClasses, train, test)
#processed_train = vec_preprocess_text(train.text.values)
#processed_test = vec_preprocess_text(test.text.values)
#print(processed_train[0])
#print(processed_test[0])
# get train and test labels
encoder = LabelEncoder()
i = 0
for data in trainLoader:
    if i == 0:
        print(data)
        print(len(data))
    i = i + 1
#labels = [label for _,label in trainLoader]
#print(labels)
encoded_labels_train = encoder.fit_transform([str(label) for _,label in trainLoader])
encoded_labels_test = encoder.fit_transform([str(label) for _,label in testLoader])
# map train and test tokens to ids
maxLen = 32
#processed_train = [list(tokenize(processed_train[i])) for i in range(processed_train.shape[0])]
#idsTrn = token2id(processed_train, maxLen, w2vModel)
#idsTrn = token2idNew([str(string_) for string_,_ in train], maxLen, vocabIterator,en_tokenizer)
#processed_test = [list(tokenize(processed_test[i])) for i in range(processed_test.shape[0])]
#idsTst = token2id(processed_test, maxLen, w2vModel)
#idsTst = token2idNew([str(string_) for string_,_ in test],maxLen, vocabIterator,en_tokenizer)


[('crock pot ladies crock pot apple pie moonshine', 'mom maple apple pie recipe taste of home', 'cookin canuck baked apple pie egg rolls recipe', 'dutch apple pie recipe just pinch recipes', 'our share of the harvest raquo grandma apple pie with oat crumb topping'), ('apple_pie', 'apple_pie', 'apple_pie', 'apple_pie', 'apple_pie')]
2


In [27]:
class w2nModel(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses, padId):
        super(w2nModel, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size,batch_first=True)
        #self.bn = nn.BatchNorm1d(hidden_size)
        self.drop1 = nn.Dropout(p = 0.5)
        self.FC1 = nn.Linear(hidden_size,256)
        self.drop2 = nn.Dropout(p = 0.5)
        self.FC2 = nn.Linear(256,nClasses)
        self.Relu = torch.nn.ReLU()
        self.padId = padId


    def forward(self, x_):
        x = self.word_embeddings(x_) # input dimensions are (batch size, sqeuenquence length) - output dimensions are (batch size, sequence length, feature size)
        x[x_ == self.padId,:] = 0 # set rows, where each row is of dimension equal to feature size, corresponding to pad id = 0
        (x,_) = self.LSTM(x)        
        x = x[:, -1, :] # gives two dimensional output, not three dimensional output -  we are retaining the last hidden state                        
        x = self.drop1(x)               
        x = self.Relu(self.FC1(x))
        x = self.drop2(x)
        x = self.FC2(x)
        return x

### Train Neural Net

In [28]:
import numpy
torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
hidden_size = 64
batchSize = 512
padId = vocabIterator.get_stoi()["<pad>"]
dModel = 100
vocabSize = (vocabIterator.__len__() )
model = w2nModel(vocab_size = vocabSize, 
                 embedding_dim = dModel,
                 hidden_size=hidden_size, nClasses = numClasses, padId = padId
                )

#labels_train = torch.tensor([int(encoded_labels_train[i]) for i in range(encoded_labels_train.shape[0])]).long()
labels_train = torch.tensor([int(encoded_labels_train[i]) for i in range(encoded_labels_train.shape[0])]).long()
trainloader = torch.utils.data.DataLoader(list(zip(token2idNew( [str(string_) for string_,_ in trainLoader], maxLen, vocabIterator,en_tokenizer).int(),
                                                labels_train)
                                                ),
                                           batch_size=batchSize,
                                         shuffle=True)

#trainLoader = DataLoader(trainData,batch_size=batchSize,shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.005)
lossVals = []
model = model.to(device)
print(model)

cpu
Pad Id = 0
w2nModel(
  (word_embeddings): Embedding(19548, 100)
  (LSTM): LSTM(100, 64, batch_first=True)
  (drop1): Dropout(p=0.5, inplace=False)
  (FC1): Linear(in_features=64, out_features=256, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (FC2): Linear(in_features=256, out_features=101, bias=True)
  (Relu): ReLU()
)


In [38]:
for epoch in range(100):  # loop over the dataset multiple times
    for data in trainLoader:
        inputs = data[0][:]
        labels = data[1][:]
        print(data[0][:2])
        #print(labels.shape)
        #inputs = inputs.to(device) 
        #labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        lossVals.append(loss.detach().clone().cpu())
        optimizer.step()
    if epoch%5 == 0:
        print(loss)
print('Finished Training')

('crock pot ladies crock pot apple pie moonshine', 'mom maple apple pie recipe taste of home')


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not tuple

In [ ]:
labels_test = torch.tensor([int(encoded_labels_test[i]) for i in range(encoded_labels_test.shape[0])]).long()
#labels_train = torch.tensor([int(encoded_labels_train[i]) for i in range(encoded_labels_train.shape[0])]).long()

testLoader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
# again no gradients needed
model.eval()
correct_pred = 0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred += 1

accuracy = 100 * float(correct_pred)/ idsTst.shape[0]
print(f"Classification Accuracy is {accuracy}%")

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=1)
ax.plot(range(len(lossVals)),lossVals)
ax.set_xlabel('Iterations',fontsize = 15)
ax.set_ylabel('Cross Entropy Loss', fontsize = 15)
ax.set_title('Classification Accuracy = {:.2f}%'.format(accuracy),fontsize = 15)
path = 'dim_{}_accry_{:.2f}len_{}_hidden_{}'.format(dim,accuracy, maxLen,hidden_size)
plt.savefig(path+'.pdf')

### Get 5 Most Error-Full Classes

In [ ]:
path = os.getcwd()
checkpoint_path = path+'\_accrcy_{:.2f}_dim_{}.pt'.format(accuracy,dim)
print(checkpoint_path)
torch.save(model, checkpoint_path)

# SVM with LSTM

### Declare LSTM model

In [ ]:
class w2nModelSVM(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses):
        super(w2nModelSVM, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size,batch_first=True)

    def forward(self, x):
        x = self.word_embeddings(x)     # output dimensions is batch size = N x sequence length x feature size
        (x,_) = self.LSTM(x)        
        x = x[:, -1, :]                 # gives two dimensional output, not three dimensional output
        return x

torch.device('cuda:0')
modelSVM = w2nModelSVM(vocab_size = w2v_weights.shape[0], 
                 embedding_dim = w2v_weights.shape[1],
                 hidden_size=hidden_size, nClasses = numClasses
                )
modelSVM.word_embeddings.weight.data.copy_(torch.from_numpy(w2v_weights))
modelSVM.word_embeddings.weight.requires_grad=False
modelSVM.LSTM.load_state_dict(model.LSTM.state_dict())
modelSVM.LSTM.requires_grad_=False

### Get LSTM Embeddings for Train and Test Dataset

In [ ]:
labels_train = torch.tensor(encoded_labels_train).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTrn.int(), labels_train)), batch_size=batchSize,
                                         shuffle=True)
trnEmbdngs = np.zeros((idsTrn.shape[0],hidden_size))
trnLbls  = np.zeros((idsTrn.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelSVM(inputs)
    trnEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    trnLbls[i*batchSize: (i+1)*batchSize] = labels

labels_test = torch.tensor(encoded_labels_test).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
tstEmbdngs = np.zeros((idsTst.shape[0],hidden_size))
tstLbls  = np.zeros((idsTst.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelSVM(inputs)
    tstEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    tstLbls[i*batchSize: (i+1)*batchSize] = labels

### Linear SVM


In [ ]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo', kernel = 'linear')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyLnr = clf.score(trnEmbdngs, trnLbls)
TstAccrcyLnr = clf.score(tstEmbdngs,tstLbls)
print(r'Train Accuracy of Linear SVM =', TrnAccrcyLnr)
print(TstAccrcyLnr)

### RBF Kernel SVM

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel='rbf')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyKernel = clf.score(trnEmbdngs, trnLbls)
TstAccrcyKernel = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyKernel)
print(TstAccrcyKernel)

# SVM without LSTM - Averaging Word2Vec Embeddings

In [ ]:
class w2nModelSVM(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses):
        super(w2nModelSVM, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        x = self.word_embeddings(x)                # output dimensions is batch size = N x sequence length x feature size
        x = torch.sum(x,dim=1)/x.shape[1]          # batch size x feature size
        return x

torch.device('cuda:0')
modelw2vAvg = w2nModelSVM(vocab_size = w2v_weights.shape[0], 
                 embedding_dim = w2v_weights.shape[1],
                 hidden_size=hidden_size, nClasses = numClasses
                )
modelw2vAvg.word_embeddings.weight.data.copy_(torch.from_numpy(w2v_weights))
modelw2vAvg.word_embeddings.weight.requires_grad=False


In [ ]:
labels_train = torch.tensor(encoded_labels_train).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTrn.int(), labels_train)),
                                         batch_size=batchSize,
                                         shuffle=True)
trnEmbdngs = np.zeros((idsTrn.shape[0],dim))
trnLbls  = np.zeros((idsTrn.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelw2vAvg(inputs)
    trnEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    trnLbls[i*batchSize: (i+1)*batchSize] = labels

labels_test = torch.tensor(encoded_labels_test).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
tstEmbdngs = np.zeros((idsTst.shape[0],dim))
tstLbls  = np.zeros((idsTst.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelw2vAvg(inputs)
    tstEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    tstLbls[i*batchSize: (i+1)*batchSize] = labels

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel = 'linear')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyLnr = clf.score(trnEmbdngs, trnLbls)
TstAccrcyLnr = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyLnr)
print(TstAccrcyLnr)

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel='rbf')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyKernel = clf.score(trnEmbdngs, trnLbls)
TstAccrcyKernel = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyKernel)
print(TstAccrcyKernel)